In [1]:
import json

import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df_elections = pd.read_csv('/mnt/data/_results/asset_evolution_elections_2018.csv')

In [3]:
# Filtra por últimas eleições
df_elections = df_elections[df_elections.year.isin([2010, 2014, 2018])]

In [4]:
# Selectiona candidatos que concorreram ao menos em 2 eleições
counts = df_elections.name.value_counts()
candidates = counts[counts>=2].index

In [5]:
# Separa o dataframe apenas usando candidatos de ao menos 2 eleições
df = df_elections[df_elections.name.isin(candidates)]
df.head(2)

,name,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared,Unnamed: 9
2012,FRANCISCO OSMAR DIÓGENES BAQUIT,3,2018,CE,PDT,DEPUTADO(A) ESTADUAL,0,1041879.42,6.0,NaN
2012,FRANCISCO OSMAR DIÓGENES BAQUIT,3,2014,CE,PSD,DEPUTADO(A) ESTADUAL,ELEITO POR QP,150000.0,1.0,NaN


In [6]:
df.shape

(6167, 10)

In [7]:
# Remove lugares onde temos None
df.sum_of_assets_declared = df.sum_of_assets_declared.str.replace('None', '0')

In [8]:
# Substitui valores ausentes por 0
df.sum_of_assets_declared.fillna(0, inplace=True)

# Muda coluna de soma de bens para decimal
df.sum_of_assets_declared = df.sum_of_assets_declared.astype(float)

In [9]:
# Transformando em milhoes para facilitar
df.sum_of_assets_declared = df.sum_of_assets_declared / 10**6

In [10]:
df.head()

,name,num_of_elections_run,year,state,party,position,result,sum_of_assets_declared,num_of_assets_declared,Unnamed: 9
2012,FRANCISCO OSMAR DIÓGENES BAQUIT,3,2018,CE,PDT,DEPUTADO(A) ESTADUAL,0,1.041879,6.0,NaN
2012,FRANCISCO OSMAR DIÓGENES BAQUIT,3,2014,CE,PSD,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.150000,1.0,NaN
60430,FRANCISCO DAS CHAGAS GONZAGA,10,2018,CE,PSTU,GOVERNADOR(A),0,0.146000,4.0,NaN
60430,FRANCISCO DAS CHAGAS GONZAGA,10,2010,CE,PSTU,GOVERNADOR(A),NÃO ELEITO,0.025000,1.0,NaN
60430,FRANCISCO DAS CHAGAS GONZAGA,10,2014,CE,PSTU,DEPUTADO(A) FEDERAL,NÃO ELEITO,0.115000,2.0,NaN


# Análise por pessoa

In [11]:
df.position = pd.Categorical(df.position,
                             categories=['PRESIDENTE(A) DA REPÚBLICA',  'GOVERNADOR(A)', 'SENADOR(A)', 
                                         'DEPUTADO(A) FEDERAL', 'DEPUTADO(A) ESTADUAL',
                                         'VICE-GOVERNADOR(A)', '2º SUPLENTE', 
                                         '1º SUPLENTE SENADOR(A)', '1º SUPLENTE', '2º SUPLENTE SENADOR(A)', 
                                         'VICE-PRESIDENTE(A)', 'DEPUTADO(A) DISTRITAL'][::-1],
                             ordered=True)

In [12]:
df.result = pd.Categorical(df.result, 
                           categories=['ELEITO', 'ELEITO POR QP', 'NÃO ELEITO', 
                                       'REJEITADO', 'SUPLENTE', 'SUBSTITUÍDO', 
                                       '2o TURNO','0'][::-1],
                           ordered=True)

In [13]:
df_result = df.groupby(['name', 'year'])['result'].max().unstack().reset_index()
df_result.rename(columns={
    2010.0: '2010-resultado',
    2014.0: '2014-resultado',
    2018.0: '2018-resultado',
}, inplace=True)

del df_result.columns.name
df_result.set_index('name', inplace=True)
df_result.reset_index()

df_result.drop(['2018-resultado'], axis=True, inplace=True)
df_result.head()

,2010-resultado,2014-resultado
name,,
ABDALA HABIB FRAXE JUNIOR,ELEITO,ELEITO POR QP
ABEILARD GOULART DE SOUZA FILHO,NaN,SUPLENTE
ABEL DO PORTO SANTOS,NÃO ELEITO,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NÃO ELEITO
ABIDENE SALUSTIANO DA SILVA,NaN,SUPLENTE


In [14]:
df_position = df.groupby(['name', 'year'])['position'].max().unstack().reset_index()
df_position.rename(columns={
    2010.0: '2010-cargo',
    2014.0: '2014-cargo',
    2018.0: '2018-cargo',
}, inplace=True)

del df_position.columns.name
df_position.set_index('name', inplace=True)
df_position.reset_index()
df_position.head()

,2010-cargo,2014-cargo,2018-cargo
name,,,
ABDALA HABIB FRAXE JUNIOR,DEPUTADO(A) ESTADUAL,DEPUTADO(A) ESTADUAL,DEPUTADO(A) ESTADUAL
ABEILARD GOULART DE SOUZA FILHO,NaN,DEPUTADO(A) ESTADUAL,DEPUTADO(A) FEDERAL
ABEL DO PORTO SANTOS,DEPUTADO(A) FEDERAL,NaN,DEPUTADO(A) FEDERAL
ABEL MOACIR BURGDURFF DE MORAES,NaN,1º SUPLENTE,DEPUTADO(A) ESTADUAL
ABIDENE SALUSTIANO DA SILVA,NaN,DEPUTADO(A) FEDERAL,DEPUTADO(A) ESTADUAL


In [15]:
df_assets = df.groupby(['name', 'year'])['sum_of_assets_declared'].max().unstack().reset_index()
df_assets.rename(columns={
    2010.0: '2010-soma-bens',
    2014.0: '2014-soma-bens',
    2018.0: '2018-soma-bens',
}, inplace=True)


del df_assets.columns.name
df_assets.set_index('name', inplace=True)
df_assets.reset_index()
df_assets.head()

,2010-soma-bens,2014-soma-bens,2018-soma-bens
name,,,
ABDALA HABIB FRAXE JUNIOR,0.550,0.550000,1.100000
ABEILARD GOULART DE SOUZA FILHO,NaN,0.112152,0.304596
ABEL DO PORTO SANTOS,0.035,NaN,0.050000
ABEL MOACIR BURGDURFF DE MORAES,NaN,0.005000,0.606000
ABIDENE SALUSTIANO DA SILVA,NaN,0.028000,0.270000


In [16]:
dd = pd.concat([df_result, df_position, df_assets], axis=1)

dd['2010-2014'] = dd['2014-soma-bens'] - dd['2010-soma-bens']
dd['2010-2014-percent'] = ((dd['2014-soma-bens'] - dd['2010-soma-bens']) / dd['2010-soma-bens'])*100
dd['2014-2018'] = dd['2018-soma-bens'] - dd['2014-soma-bens']
dd['2014-2018-percent'] = ((dd['2018-soma-bens'] - dd['2014-soma-bens']) / dd['2014-soma-bens'])*100
dd['2010-2018'] = dd['2018-soma-bens'] - dd['2010-soma-bens']
dd['2010-2018-percent'] = ((dd['2018-soma-bens'] - dd['2010-soma-bens']) / dd['2010-soma-bens'])*100


#dd['2010-2014-percent-pretty'] = dd['2010-2014-percent'].apply(lambda x: '{:.1f}%'.format(x))
dd['2014-2018-percent-pretty'] = dd['2014-2018-percent'].apply(lambda x: '{:.1f}%'.format(x))
dd['2010-2018-percent-pretty'] = dd['2010-2018-percent'].apply(lambda x: '{:.1f}%'.format(x))

df_final = dd[[
    #'2010-cargo', '2010-resultado', '2010-soma-bens', 
    '2014-cargo', '2014-resultado', '2014-soma-bens', #'2010-2014', '2010-2014-percent', '2010-2014-percent-pretty',
    '2018-cargo', '2018-soma-bens', '2014-2018', '2014-2018-percent', '2014-2018-percent-pretty', 
    #'2010-2018', '2010-2018-percent', '2010-2018-percent-pretty',
]]

In [17]:
drop = [
    #'2010-2014-percent', 
    '2014-2018-percent', 
    #'2010-2018-percent'
]

In [18]:
top30_2018 = df_final.sort_values(by='2018-soma-bens', ascending=False).head(30)
top30_2018.drop(drop, axis=1, inplace=True)
top30_2018.to_csv('/mnt/data/_results/top-30-ricos-nas-eleicoes-2018.csv')
top30_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
OTAVIANO OLAVO PIVETTA,NaN,NaN,NaN,VICE-GOVERNADOR(A),758.912854,NaN,nan%
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.619703,DEPUTADO(A) ESTADUAL,307.438779,306.819076,49510.7%
PEDRO CHAVES DOS SANTOS FILHO,NaN,NaN,NaN,SENADOR(A),260.989122,NaN,nan%
MAURO MENDES FERREIRA,NaN,NaN,NaN,GOVERNADOR(A),226.907614,NaN,nan%
IVONE MEZZOMO CASSOL,SENADOR(A),NÃO ELEITO,56.276546,1º SUPLENTE,169.428539,113.151993,201.1%
NILSO JOSÉ BERLANDA,DEPUTADO(A) ESTADUAL,SUPLENTE,6.266501,DEPUTADO(A) ESTADUAL,129.618954,123.352453,1968.4%
REINALDO AZAMBUJA SILVA,GOVERNADOR(A),ELEITO,27.440416,GOVERNADOR(A),77.397395,49.956979,182.1%
DÂMINA DE CARVALHO PEREIRA,DEPUTADO(A) FEDERAL,ELEITO POR QP,38.827718,DEPUTADO(A) FEDERAL,75.173523,36.345805,93.6%
EDENILSO ROSSI ARNALDI,DEPUTADO(A) FEDERAL,0,27.077399,DEPUTADO(A) FEDERAL,67.179340,40.101941,148.1%


In [19]:
top30_crescimento_absoluto_2018 = df_final.sort_values(by='2014-2018', ascending=False).head(30)
top30_crescimento_absoluto_2018.drop(drop, axis=1, inplace=True)
top30_crescimento_absoluto_2018.to_csv(
    '/mnt/data/_results/top-30-que-enriqueceram-desde-as-eleicoes-2014-valor-absoluto.csv')
top30_crescimento_absoluto_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,0.619703,DEPUTADO(A) ESTADUAL,307.438779,306.819076,49510.7%
NILSO JOSÉ BERLANDA,DEPUTADO(A) ESTADUAL,SUPLENTE,6.266501,DEPUTADO(A) ESTADUAL,129.618954,123.352453,1968.4%
IVONE MEZZOMO CASSOL,SENADOR(A),NÃO ELEITO,56.276546,1º SUPLENTE,169.428539,113.151993,201.1%
MARINALDO ROSENDO DE ALBUQUERQUE,DEPUTADO(A) FEDERAL,ELEITO POR QP,13.026175,DEPUTADO(A) FEDERAL,64.470224,51.444049,394.9%
REINALDO AZAMBUJA SILVA,GOVERNADOR(A),ELEITO,27.440416,GOVERNADOR(A),77.397395,49.956979,182.1%
MIGUEL MOUBADDA HADDAD,DEPUTADO(A) FEDERAL,ELEITO POR QP,11.383318,DEPUTADO(A) FEDERAL,58.633150,47.249832,415.1%
DILMAR DAL BOSCO,DEPUTADO(A) ESTADUAL,ELEITO POR QP,2.394865,DEPUTADO(A) ESTADUAL,44.504337,42.109472,1758.3%
EDENILSO ROSSI ARNALDI,DEPUTADO(A) FEDERAL,0,27.077399,DEPUTADO(A) FEDERAL,67.179340,40.101941,148.1%
DÂMINA DE CARVALHO PEREIRA,DEPUTADO(A) FEDERAL,ELEITO POR QP,38.827718,DEPUTADO(A) FEDERAL,75.173523,36.345805,93.6%


In [20]:
df_sem_percent_infinito = df_final[df_final['2014-2018-percent']!= np.inf]
top30_crescimento_percent_2018 = df_sem_percent_infinito.sort_values(by='2014-2018-percent', ascending=False).head(30)
top30_crescimento_percent_2018.drop(drop, axis=1, inplace=True)
top30_crescimento_percent_2018.to_csv(
    '/mnt/data/_results/top-30-que-enriqueceram-desde-as-eleicoes-2014-valor-percent.csv')

top30_crescimento_percent_2018.head(10)

,2014-cargo,2014-resultado,2014-soma-bens,2018-cargo,2018-soma-bens,2014-2018,2014-2018-percent-pretty
name,,,,,,,
RUBENS SANCHEZ PROENÇA,DEPUTADO(A) FEDERAL,0,3.000000e-08,DEPUTADO(A) FEDERAL,0.024000,0.024000,79999900.0%
MARDEN LUIS BRITO CAVALCANTE E MENESES,DEPUTADO(A) ESTADUAL,ELEITO POR QP,6.197028e-01,DEPUTADO(A) ESTADUAL,307.438779,306.819076,49510.7%
CARLOS DO SOCORRO GUERREIRO VAZ,DEPUTADO(A) FEDERAL,SUPLENTE,3.000000e-03,DEPUTADO(A) FEDERAL,1.014000,1.011000,33700.0%
CLEINALDO CASTRO LOPES,DEPUTADO(A) ESTADUAL,SUPLENTE,3.500000e-03,DEPUTADO(A) ESTADUAL,0.738970,0.735470,21013.4%
NAIARA LOURENÇO,DEPUTADO(A) DISTRITAL,NÃO ELEITO,2.000000e-02,DEPUTADO(A) FEDERAL,4.092000,4.072000,20360.0%
GUSTAVO DE BIASE NUNES DA ROCHA,DEPUTADO(A) FEDERAL,SUPLENTE,2.500000e-03,DEPUTADO(A) FEDERAL,0.342000,0.339500,13580.0%
BESALIEL DE OLIVEIRA RODRIGUES,DEPUTADO(A) ESTADUAL,NÃO ELEITO,1.300000e-02,DEPUTADO(A) ESTADUAL,1.760000,1.747000,13438.5%
ABEL MOACIR BURGDURFF DE MORAES,1º SUPLENTE,NÃO ELEITO,5.000000e-03,DEPUTADO(A) ESTADUAL,0.606000,0.601000,12020.0%
SILVIO LUIZ RODRIGUES RAMALHO,DEPUTADO(A) ESTADUAL,SUPLENTE,5.000000e-03,DEPUTADO(A) ESTADUAL,0.488000,0.483000,9660.0%


# Análise de Evolução Patrimonial por Partido

In [21]:
df.party = pd.Categorical(df.party,
                             categories=['PDT', 'PSD', 'PSTU', 'PV', 'PSDB', 'PMDB', 'PROS', 'PSOL', 'PP',
                                         'PRP', 'PSB', 'PT', 'PSC', 'PRB', 'PMN', 'PR', 'PPS', 'PTC',
                                         'PODE', 'PSL', 'PRTB', 'PTN', 'PSDC', 'DEM', 'PMB', 'PHS', 'SD',
                                         'REDE', 'PPL', 'PTB', 'PEN', 'PCB', 'NOVO', 'PCO'],
                             ordered=True)

In [22]:
df_evolution = dd[[
    '2010-2014', '2010-2014-percent',
    '2014-2018', '2014-2018-percent',
]]
df_evolution.head()

,2010-2014,2010-2014-percent,2014-2018,2014-2018-percent
name,,,,
ABDALA HABIB FRAXE JUNIOR,0.0,0.0,0.550000,100.000000
ABEILARD GOULART DE SOUZA FILHO,NaN,NaN,0.192444,171.591321
ABEL DO PORTO SANTOS,NaN,NaN,NaN,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NaN,0.601000,12020.000000
ABIDENE SALUSTIANO DA SILVA,NaN,NaN,0.242000,864.285714


In [23]:
df_party = df.groupby(['name', 'year'])['party'].max().unstack().reset_index()
df_party.rename(columns={
    2010.0: '2010-partido',
    2014.0: '2014-partido',
    2018.0: '2018-partido',
}, inplace=True)

del df_party.columns.name
df_party.set_index('name', inplace=True)
df_party.reset_index()

df_party.head()

,2010-partido,2014-partido,2018-partido
name,,,
ABDALA HABIB FRAXE JUNIOR,PTN,PTN,PODE
ABEILARD GOULART DE SOUZA FILHO,NaN,SD,PDT
ABEL DO PORTO SANTOS,PRTB,NaN,PRB
ABEL MOACIR BURGDURFF DE MORAES,NaN,PSTU,PSTU
ABIDENE SALUSTIANO DA SILVA,NaN,PSC,PSC


In [24]:
dd = pd.concat([df_evolution, df_party], axis=1, sort=False)
dd = dd[['2010-partido', '2010-2014', '2010-2014-percent', '2014-partido', '2014-2018', '2014-2018-percent']]
dd.head()

,2010-partido,2010-2014,2010-2014-percent,2014-partido,2014-2018,2014-2018-percent
name,,,,,,
ABDALA HABIB FRAXE JUNIOR,PTN,0.0,0.0,PTN,0.550000,100.000000
ABEILARD GOULART DE SOUZA FILHO,NaN,NaN,NaN,SD,0.192444,171.591321
ABEL DO PORTO SANTOS,PRTB,NaN,NaN,NaN,NaN,NaN
ABEL MOACIR BURGDURFF DE MORAES,NaN,NaN,NaN,PSTU,0.601000,12020.000000
ABIDENE SALUSTIANO DA SILVA,NaN,NaN,NaN,PSC,0.242000,864.285714


In [25]:
party_evolution_2010 = dd.groupby(['2010-partido'])['2010-2014'].sum()
party_count_2010 = dd.groupby(['2010-partido'])['2010-2014'].count()


evolution_2010_proportional = party_evolution_2010/party_count_2010 
evolution_2010_proportional_sorted = evolution_2010_proportional.sort_values(ascending=False)[0:10]
evolution_2010_proportional_sorted.to_csv('/mnt/data/_results/top-10-partidos-que-mais-enriqueceram-2010a2014.csv')

evolution_2010_proportional_sorted

2010-partido
PR      1.059006
DEM     0.762060
PSDB    0.611865
PSDC    0.584179
PP      0.581336
PMN     0.542390
PTB     0.521047
PSC     0.489934
PV      0.478101
PDT     0.364981
Name: 2010-2014, dtype: float64

In [26]:
party_evolution_2014 = dd.groupby(['2014-partido'])['2014-2018'].sum()
party_count_2014 = dd.groupby(['2014-partido'])['2014-2018'].count()

evolution_2014_proportional = party_evolution_2014/party_count_2014
evolution_2014_proportional_sorted = evolution_2014_proportional.sort_values(ascending=False)[0:10]
evolution_2014_proportional_sorted.to_csv('/mnt/data/_results/top-10-partidos-que-mais-enriqueceram-2014a2018.csv')

evolution_2014_proportional_sorted

2014-partido
DEM     5.036606
PSDB    4.251273
PSD     3.009029
PR      2.901780
PP      2.597425
PSB     2.132422
PROS    1.956783
PMDB    1.737038
PMN     1.382670
PDT     1.226111
Name: 2014-2018, dtype: float64

# Análise de Declaração Patrimonial por Partido

In [27]:
assets_2018 = pd.concat([df_party['2018-partido'], df_final['2018-soma-bens']], axis=1)

In [28]:
party_assets_sum = assets_2018.groupby(['2018-partido'])['2018-soma-bens'].sum()
party_person_count = assets_2018.groupby(['2018-partido'])['2018-soma-bens'].count()

party_declared_mean_asset_values = party_assets_sum / party_person_count
party_declared_mean_asset_values_ordered = party_declared_mean_asset_values.sort_values(ascending=False)[0:10]

party_declared_mean_asset_values_ordered.to_csv('/mnt/data/_results/top-10-partidos-que-ricos-eleicoes-2018.csv')
party_declared_mean_asset_values_ordered

2018-partido
PDT     7.930143
DEM     7.200384
PSDB    6.154240
PP      5.681493
NOVO    5.261831
PR      4.726481
PSD     4.265588
PRB     3.719373
PODE    2.269670
PSB     1.929101
Name: 2018-soma-bens, dtype: float64